In [13]:
# kakao blind 2022 2차 코딩테스트
# 최성인

import requests
import json

x_auth_token = 'd74af3d99f4be7c9450f4a62859980ad'
base_url = 'https://huqeyhi95c.execute-api.ap-northeast-2.amazonaws.com/prod'


header = {'X-Auth-Token':x_auth_token, 'Content-Type': 'application/json'}
data = {'problem':1} # 2
num_users = 31 #901
rank_diff = 3000 
th_time = 11

res = requests.post(base_url+"/start",data=json.dumps(data), headers=header)

auth_key = res.json()['auth_key']


In [14]:
def getInfo(inp):
    # wating_line
    # user_info
    # game_result
    h = {'Authorization':auth_key, 'Content-Type': 'application/json' }
    return requests.get(base_url+"/"+inp,headers= h).json()

def match(pairs):
    h = {'Authorization':auth_key, 'Content-Type': 'application/json' }
    data = {'pairs':pairs}
    res = requests.put(base_url+"/match",data=json.dumps(data), headers=h)
    return res.json()
def changeGrade(cmd):
    h = {'Authorization':auth_key, 'Content-Type': 'application/json' }
    data = {'commands':cmd}
    res = requests.put(base_url+"/change_grade",data=json.dumps(data), headers=h)
    return res.json()

def make_grade(id,grade):
    return {'id':id,'grade':grade}

In [15]:
def setGrade():
    cmd = []
    for i in range(1,num_users):
        cmd.append({'id':i,'grade':grade[i]})
    changeGrade(cmd)

#init
init_rank = 4000
abuse_th = 5

cmd = []
for i in range(1,num_users):
    cmd.append({'id':i,'grade':init_rank})
changeGrade(cmd)

grade = [0] * (num_users)
match_count = [0] * num_users
abusing = [0] * num_users

matched = [[0] * num_users for _ in range(num_users)]

for v in getInfo("user_info")['user_info']:
    grade[v['id']] = v['grade']


In [16]:
def nearest(rank, wl,visited, p_id):
    min_v = 10000
    t_id = 0
    # 50 이하라면 매칭수가 적은 순서로 매칭
    nears = list(filter(lambda x: abs(grade[x['id']] - rank)<50, wl))
    if len(nears) != 0 :
        return (min(nears,key=lambda x : matched[ x['id']][p_id]))['id']
    #가장 점수가 가까운 순서로 매칭
    for w in wl:
        id = w['id']
        if visited[id]:
            continue
        if min_v > abs(rank - grade[id]):
            t_id = id
            min_v = abs(rank - grade[id])
    return t_id
    
def matching():
    global cur
    match_list = []
    wl = getInfo('waiting_line')['waiting_line']
    wl = sorted(wl,key= lambda x: -(cur - x['from']))
    visited = [False]*num_users
    while wl:
        p = wl.pop()
        if visited[p['id']]:
            continue
        visited[p['id']] = True
        t_id = nearest(grade[p['id']],wl,visited,p['id'])
        #if (cur - p['from']) < th_time and abs(grade[t_id] - grade[p['id']]) > rank_diff:
        #  continue
        visited[t_id] = True
        match_list.append([p['id'],t_id])
        match_count[p['id']]+=1
        match_count[t_id]+=1
        matched[t_id][p['id']] += 1
        matched[p['id']][t_id] += 1
        
    cur = match(match_list)['time']
    
def grading():
    res = getInfo("game_result")['game_result']
    if len(res)==0:
        return
    
    for r in res:
        win = r['win']
        lose = r['lose']
        t_time = r['taken']
        if 3<= t_time <= 10:
            abusing[lose] += 1 #어뷰징 체크
                    
        grade_gap = abs(grade[win]-grade[lose]) // 10
        if grade_gap == 0:
            grade_gap = 1
        lp = int((40 - t_time) * (9900 / 35) / (grade_gap)) 
        if abusing[lose] > abuse_th or abusing[win] > abuse_th: #어뷰징인 경우 lp 반영x
            continue
        else:
            grade[lose] -= lp
        grade[win] += lp
        
        grade[win] = min(grade[win],9999)
        grade[lose] = max(grade[lose],0)

In [17]:
from tqdm import tqdm
for _ in tqdm(range(595)):
    matching()
    grading()
setGrade()
match([])

 32%|███▏      | 191/595 [01:58<03:49,  1.76it/s]

In [ ]:
getInfo("score")

{'status': 'finished',
 'efficiency_score': '94.2313',
 'accuracy_score1': '60.0',
 'accuracy_score2': '59.0558',
 'score': '218.252'}

In [ ]:
import numpy as np
print(np.std(grade))
print(np.mean(grade))

2085.2178674255106
4034.8387096774195


In [ ]:
list(filter(lambda x : x > 0 , abusing))

[]

In [ ]:
np.mean(match_count)

29.483870967741936